# **VIRTUAL RACE ENGINEER**

In [8]:
import fastf1 as ff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1.plotting
fastf1.plotting.setup_mpl()
import fastf1.api as fap
fastf1.Cache.enable_cache(r"D:\Prabhu\SEM 7\F1 Data")
import re
import datetime as dt
import fastf1.mvapi as famp


### LOADING SESSION

In [9]:
session = ff.get_session(2023,'Dutch','R')
session.load()

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '10', '11', '55', '44', '4', '23', '81', '31', '18', '27', '40', '77', '22', '20', '63', '24', '16', '2']


In [10]:
laps=session.laps
drivers=laps['Driver'].unique()

### FETCHING TELEMETRY DATA

In [11]:
total_telemetry=[]
for i in drivers:
    drivers_laps=laps.pick_driver(i)
    driver_telemetry=drivers_laps.get_car_data()

    driver_telemetry['Driver']=i
    total_telemetry.append(driver_telemetry)
telemetry=pd.concat(total_telemetry,ignore_index=True)


### FETCHING WEATHER,TYRE,LAP AND POSITION DATA 

In [12]:
weather_data=pd.DataFrame(fap.weather_data(session.api_path))
tyre_data = laps[['Driver', 'LapNumber', 'Compound', 'FreshTyre','TyreLife']]
lap_data=laps[['Driver','LapNumber','LapTime','Sector1Time','Sector2Time','Sector3Time','IsAccurate']]
position_data=laps[['Driver','LapNumber','Position']]

req            INFO 	Using cached data for weather_data


### FETCHING RACE CONTROL MESSAGES

In [13]:
rcm_data = pd.DataFrame(fap.race_control_messages(session.api_path))
msg=rcm_data[['Time','Status','Message']]

#Filter for Yellow, Red, VSC, SC 
yellow_flags_key=['YELLOW','DOUBLE YELLOW']
red_flags_key=['RED']
vsc_key=['VSC','VIRTUAL SAFETY CAR']
sc_key=['SC','SAFETY CAR']

#Searching using the keyword in REGEX
yellow_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in yellow_flags_key))]
red_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in red_flags_key))]
vsc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in vsc_key))]
sc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in sc_key))]


req            INFO 	Using cached data for race_control_messages


### FETCHING LAP DATA (PITS STOPS, DRIVER POSITION ETC.)

In [14]:
lapsdata, streamdata = fap.timing_data(session.api_path)
# pd.set_option('display.max_columns', None)
lapsdata=pd.DataFrame(lapsdata)
streamdata=pd.DataFrame(streamdata)
combined=pd.concat([lapsdata,streamdata])
laps_data=combined[['Driver','LapTime','NumberOfLaps','NumberOfPitStops','PitOutTime','PitInTime','Position','GapToLeader','IntervalToPositionAhead']]
laps_data['PitInTimeSec']=laps_data['PitInTime'].dt.total_seconds()
laps_data['PitOutTimeSec']=laps_data['PitOutTime'].dt.total_seconds()
laps_data

req            INFO 	Using cached data for _extended_timing_data
C:\Users\prabh\AppData\Local\Temp\ipykernel_10616\910154449.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_data['PitInTimeSec']=laps_data['PitInTime'].dt.total_seconds()
C:\Users\prabh\AppData\Local\Temp\ipykernel_10616\910154449.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_data['PitOutTimeSec']=laps_data['PitOutTime'].dt.total_seconds()


,Driver,LapTime,NumberOfLaps,NumberOfPitStops,PitOutTime,PitInTime,Position,GapToLeader,IntervalToPositionAhead,PitInTimeSec,PitOutTimeSec
0,1,NaT,1.0,0.0,0 days 00:14:44.885000,NaT,NaN,NaN,NaN,NaN,884.885
1,1,0 days 00:01:49.972000,2.0,0.0,NaT,0 days 01:05:25.519000,NaN,NaN,NaN,3925.519,NaN
2,1,0 days 00:01:48.658000,3.0,1.0,0 days 01:05:45.581000,NaT,NaN,NaN,NaN,NaN,3945.581
3,1,0 days 00:01:25.251000,4.0,1.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN
4,1,0 days 00:01:23.461000,5.0,1.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29975,20,NaT,NaN,NaN,NaT,NaT,14.0,+26.521,+0.344,NaN,NaN
29976,20,NaT,NaN,NaN,NaT,NaT,14.0,+23.775,+0.305,NaN,NaN
29977,20,NaT,NaN,NaN,NaT,NaT,14.0,+26.469,+0.300,NaN,NaN
29978,20,NaT,NaN,NaN,NaT,NaT,15.0,+23.775,+0.305,NaN,NaN


### GETTING CORNERS INFO

In [15]:
circuit_info=session.get_circuit_info()
corners_data=circuit_info.corners

,X,Y,Number,Letter,Angle,Distance
0,2045.235718,6685.891602,1,,66.665211,350.418607
1,1769.605835,3831.817627,2,,-18.440573,655.386285
2,733.604797,2809.591553,3,,29.283872,828.061179
3,2716.918945,3031.772217,4,,-86.771394,1050.764030
4,4555.515137,2861.585693,5,,95.349563,1238.842252
5,6001.011719,3518.558350,6,,-70.125444,1407.243430
6,8671.674805,2879.995361,7,,-165.843124,1691.908265
7,7340.243652,117.993332,8,,108.736262,2026.845812
8,5498.077637,848.862793,9,,-1.483621,2239.786177
9,7457.498047,2120.509277,10,,-167.375656,2498.808749
